# Preprocesamiento y procesamiento de señales

El siguiente código lo que hará es primero, sacar las características y después devolverá una clase por ventana de datos, como lo que hicimos en el código anterior. Solo, que esta vez, antes de realizar esto, habrá que hacer dos cosas: 

    - Estimar W para los Filter Bank Common Spatial Patterns (FBCSP) 
    - Entrenar el clasificador

Se trabajará con ventanas de 300 muestras que se van desplazando de 100 en 100. 

En los registros session los datos que realmente nos interesan son: 
    
    - data_EEG: los datos en crudo (32electrodos x muestras).
    - data_preprocessed_EEG: datos preprocesados. No son los mismos pasos que los de este código, por lo que es mejor no utilizarlos.
    - task_EEG: las tareas asociadas a cada dato.
    
En los registros tendremos las siguientes tareas: 

    - 400: tiempo inicial sin tarea.
    - 401: preparación para la relajación.
    - 402: relajación.
    - 403: preparación para la imaginación motora.
    - 404: imaginación motora.
    - 405: preparación para las operaciones matemáticas.
    - 406: operaciones matemáticas.

De las cuales solo nos interesan la 401, 402, 403 y 404.

## Primero, se leen los registros de señales

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import os

In [2]:
class RSession:
    def __init__(self,data,task, user_id, file_name, folder):
        self.data = data
        self.task = task
        self.user_id = user_id
        self.file_name = file_name
        self.folder = folder

In [3]:
import scipy.io as sio
def read_record(rec):
    ''' read_record('./RegistrosSinProcesar/user#0091#20040101#01#reg001.mat')'''
    mat = sio.loadmat(rec)
    mdata = mat['session']
    val = mdata[0,0]
    folder = rec.split("/")[-2]
    
    rsession = RSession(val["data_EEG"], val["task_EEG"], val["conf"]["acquisition"][0,0][0][0]["user_code"][0],  val["conf"]["acquisition"][0,0][0][0]["file_name"][0], folder)
    return rsession

In [4]:
rsession = read_record('./RegistrosSinProcesar/W29-29_03_2021/W29_20210329_openloop_01.mat')
print(rsession.folder)

W29-29_03_2021


# Se realiza el filtrado de paso banda: Filter Bank Common Spatial Patterns (FBCSP)

El filtrado de una señal es un proceso (analógico o digital) que elimina ciertas componentes de una señal. 

[ESTO NO, QUITAR: 
Un filtro paso-bajo es un sistema que únicamamente permite el paso de las componentes de una señal que varían lentamente (bajas frecuencias) y elimina aquellas componentes que varían rápidamente (altas frecuencias). 
Al contrario, un filtro paso-alto es un sistema que permite únicamente el paso de las componentes de una señal que varían rápidamente (altas frecuencias) y elimina aquellas componentes que varían lentamente (bajas frecuencias).]

El Filter Bank Common Spatial Pattern es un enfoque de machine learning propuesto por [CITAR] para procesar señales EEG de la imaginería motora. FBCSP realiza 4 etapas: 

    - Filtrado frecuencial (Frequency filtering): Las señales EEG son filtradas mediante un filtro de paso de banda en múltiples bandas de frecuencia. Se utilizarán múltiples filtros de paso de banda que utilizan filtros Chebyshev [TODO PREGUNTAR POR QUÉ SE USA EN EL CÓDIGO EL FILTRO BUTTERWORTH] de tipo II de fase cero. 
    - Filtrado espacial (Spatial Filtering): Seguidamente, las características CSP (Common Spatial Pattern algorithm, el cual es efectivo en construir filtros espaciales óptimos que discriminan enter dos clases de señales EEG in BCI basadas en la imaginería motora. [4, 5]) se extraen de cada una de las bandas filtradas. Es decir, mediante el algoritmo CSP se hará un filtrado espacial. 
    - Selección de características (Feature Selection): Un algoritmo de selección de características se usa para automáticamente seleccionar pares discriminativos de bandas de frecuencia y sus caracteristicas de CSP correspendientes. 
    - Clasificación: Se usa un algoritmo de clasificación para clasificar las características CSP. 
    
FBCSP proporciona un enfoque más general porque cualquier algoritmo de machine learning y de inteligencia computacional de selección y clasificación de características (features) puede ser usado. Es característico que este enfoque solo implementa los filtros espaciales efectivos cuyos pares CSP de características son seleccionados, es decir, solo implementa un pequeño subconjunto de filtros espaciales efectivos lo cual reduce la complejidad computacional.

Filtrado espacial: 
La actividad motora (real o imaginada) causa una atenuación o incremento de la frecuencia rítmica neuronal localizada, lo cual es llamado Event-Related Desynchronization (ERD) o Event-Related Synchronization (ERS) respectivamente (CITAR PAG 6 A TUTORIAL ON EEG). El algoritmo Common Spatial Pattern (CSP) es muy bueno calculando filtros espaciales para detectar ERD y ERS. El objetivo del filtrado espacial utilizando este algoritmo es calcular características cuyas varianzas son óptimas para discriminar dos clases de señales EEG. 
Este filtrado se basa en la diagonalización simultánea de dos matrices de covarianza.  

Sea $A ∈ R^{nxn}$ una matriz, se dice que A es diagonalizable $⇔$ A es semejante a una matriz diagonal D (una matriz diagonal es una matriz cuadrada con todas sus diagonales nulas menos la diagonal principal que puede tener valores nulos o no) $⇔ ∃ P ∈ R^{nxn}$ inversible tq $A=PDP^{-1}$, o lo que es lo mismo  $P^{-1}AP=D$

Una matriz de covarianza es una matriz cuadrada que contiene la covarianza entre los elementos de un vector. Sea X un vector tq $X= \begin{bmatrix}
X_{1}\\ 
...\\
X_{n}\\
\end{bmatrix}$; la matriz de covarianza $\sum $ mide la covarianza entre cada par de variables $X_{i}$ y $X_{j}$ es decir $\sum_{ij} = Cov(X_{i},X_{j}) $ y cuando i=j entonces $\sum_{ii} = Cov(X_{i},X_{i})= Var(X_{i})$, por tanto se puede ver definitivamente a la matriz de covarianza como: 
$\sum = \begin{bmatrix}
Var(X_{1}) & Cov(X_{1},X_{2}) & ... & Cov(X_{1},X_{n})\\ 
 Cov(X_{2},X_{1})& Var(X_{2}) & ... & Cov(X_{2},X_{n})\\ 
... & ... & ... & ...\\ 
Cov(X_{n},X_{1}) & Cov(X_{n},X_{2}) & ... & Var(X_{n})
\end{bmatrix}$

Continuando con el CSP, la señal espacialmente filtrada $Z$ de un único registro $E$ es dado como: $Z=WE$; donde $E$ es una matriz $NxT$ que representa las señales EEG en bruto, N es el nº de canales, T es el nº de muestras medidas por canal; $W$ es la matriz de proyección CSP. Las filas de W son los filtros espaciales estacionarios y las columnas de $W^{-1}$ son los common spatial patterns. $Z_{p}$ tq $p \in {1..2m}$ (m primeras y últimas filas de Z, porque el filtro espacial Z maximiza la diferencia entre la varianza de dos clases de señales EEG) formará el vector de características que será la entrada del clasificador: $X_{p}= log(\frac{var(Z_{p})}{\sum_{i=1}^{2m}var(Z_{p})})$


Un bandpass (paso de banda) es el rango de frecuencias o longitudes de onda que pueden pasar a través de un filtro. Un filtro de este tipo sirve para seleccionar frecuencias deseadas. 

Butterworth filter: Es un filtro de procesamiento de la señal diseñado para tener una respuesta de frecuencia lo más plana posible en el paso de banda. 

# 1- SE CREARAN PRIMERO TODAS LAS FUNCIONES NECESARIAS

## Se instancian los parámetros globales

In [5]:
# Parámetros globales
fm = 200 # Frecuencia de Muestreo - Hz
electrodes_names_selected = ['F3', 'FZ', 'FC1','FCZ','C1','CZ','CP1','CPZ', 'FC5', 'FC3','C5','C3','CP5','CP3','P3',
                             'PZ','F4','FC2','FC4','FC6','C2','C4','CP2','CP4','C6','CP6','P4','HR' ,'HL', 'VU', 'VD']
electrodeEyes_names_selected = ['HR', 'HL', 'VU', 'VD'] # Electrodos que se colocarón en los ojos - No los utilizaremos 
number_channels = len(electrodes_names_selected)
print(number_channels)

31


## Se comienza con el filtrado frecuencial usando un filtro de paso de banda

Primero instanciamos el orden de los filtros (filt_order) y las frecuencias de corte (low_f y high_f), para poder estimar los parámetros del filtro.

In [6]:
from scipy import signal

In [7]:
# CÓDIGO PARA ESTIMAR LOS PARÁMETROS DEL FILTRO:
def butter_band_pass(fm, number_channels, filt_order, low_f, high_f):
    # Filtra las frecuencias fuera de la banda seleccionada
    # Input: original fm, number of channels, Butterworth filter order, low and high cut frequencies
    # Output: Parameters of the filter
    
    fNq = fm/2 # Por el th. de Nyquist
    Wn = [low_f, high_f]
    Wn1 = [w/fNq for w in Wn] #Frecuencia no dimensional de paso de banda de Butterworth
    
    b, a = signal.butter (filt_order, Wn1, 'bandpass') # Construimos el filtro
    return b,a

In [8]:
# BUTTERWORTH BANDPASS FILTER
def butter_bandpass_filter(data, fm, number_channels, filt_order, low_f, high_f):
    # Filtra las frecuencias fuera de la banda seleccionada
    # Input: raw data, original fm, number of channels, Butterworth filter order, low and high cut frequencies
    # Output: Signal filtered
    
    b, a = butter_band_pass(fm, number_channels, filt_order, low_f, high_f)
    y = signal.lfilter(b, a, data)
    return y

In [9]:
filt_order = 2 # Todos los filtros tienen el mismo orden. Serán filtros de 2º orden Butterworth

In [10]:
# Primer filtro: 
low_f = 5
high_f = 10
data_filter1 = butter_bandpass_filter(rsession.data, fm, number_channels, filt_order, low_f, high_f)
data_filter1

array([[ 0.05203136,  0.2272646 ,  0.52783001, ..., -5.16449605,
        -4.54777798, -3.77517685],
       [ 0.07284045,  0.32995867,  0.77647454, ..., -3.5575485 ,
        -2.82991263, -2.10169168],
       [ 0.07273468,  0.32464202,  0.74818458, ..., -2.89301525,
        -2.7038814 , -2.54436905],
       ...,
       [ 0.06353644,  0.27471246,  0.58688548, ..., -3.71160372,
        -2.94415099, -2.12780871],
       [ 0.07250528,  0.32595649,  0.74320048, ..., -5.38334973,
        -4.26979061, -2.93767953],
       [ 0.03869447,  0.13236771,  0.21634046, ..., -2.98093698,
        -2.42140039, -1.76862756]])

In [11]:
# Segundo filtro: 
low_f = 10
high_f = 15
data_filter2 = butter_bandpass_filter(rsession.data, fm, number_channels, filt_order, low_f, high_f)
data_filter2

array([[ 0.05203136,  0.21770155,  0.46980341, ...,  1.08067233,
         0.52201331, -0.12727453],
       [ 0.07284045,  0.31657104,  0.69307178, ...,  1.4315929 ,
         0.55135541, -0.52337878],
       [ 0.07273468,  0.31127383,  0.66579203, ...,  0.37071941,
        -0.55423992, -1.49224305],
       ...,
       [ 0.06353644,  0.26303485,  0.51654359, ...,  2.50829058,
         2.43914833,  1.90633776],
       [ 0.07250528,  0.31263046,  0.66063801, ...,  2.28015343,
         1.84064048,  1.19327188],
       [ 0.03869447,  0.1252559 ,  0.17992233, ...,  1.46313747,
         1.58723228,  1.43554018]])

In [12]:
# Tercer filtro: 
low_f = 15
high_f = 20
data_filter3 = butter_bandpass_filter(rsession.data, fm, number_channels, filt_order, low_f, high_f)
data_filter3

array([[ 0.05203136,  0.20365206,  0.38933143, ...,  1.11395273,
         1.33097396,  1.03061825],
       [ 0.07284045,  0.29690268,  0.57722922, ...,  1.63229941,
         0.95771042, -0.12076354],
       [ 0.07273468,  0.29163403,  0.55142391, ...,  0.70498146,
         0.50080113,  0.06122699],
       ...,
       [ 0.06353644,  0.24587876,  0.41903505, ...,  1.87054188,
         1.31765   ,  0.18295486],
       [ 0.07250528,  0.2930526 ,  0.54599919, ...,  2.66295215,
         1.68939503,  0.21172826],
       [ 0.03869447,  0.11480763,  0.12997172, ...,  3.43426277,
         1.75701094, -0.46236147]])

In [13]:
# Cuarto filtro: 
low_f = 20
high_f = 25
data_filter4 = butter_bandpass_filter(rsession.data, fm, number_channels, filt_order, low_f, high_f)
data_filter4

array([[ 0.05203136,  0.18546207,  0.29358898, ...,  2.07020221,
         0.12575692, -2.0655995 ],
       [ 0.07284045,  0.27143789,  0.43906973, ...,  0.68802516,
        -0.08322236, -1.00595018],
       [ 0.07273468,  0.26620623,  0.41515623, ...,  1.71391338,
         0.50020939, -1.0953182 ],
       ...,
       [ 0.06353644,  0.22366663,  0.30310263, ...,  1.97122517,
         0.16034405, -1.98479915],
       [ 0.07250528,  0.26770499,  0.40934381, ...,  0.8185647 ,
         0.42084132, -0.41161594],
       [ 0.03869447,  0.10128017,  0.07158081, ...,  0.03039028,
        -1.93255217, -3.10721516]])

In [14]:
data_filtered = np.concatenate((np.array(data_filter1), np.array(data_filter2)), axis = 0)
data_filtered = np.concatenate((np.array(data_filtered), np.array(data_filter3)), axis = 0)
data_filtered = np.concatenate((np.array(data_filtered), np.array(data_filter4)), axis = 0)

print(np.shape(data_filtered))
data_filtered

(124, 14600)


array([[ 0.05203136,  0.2272646 ,  0.52783001, ..., -5.16449605,
        -4.54777798, -3.77517685],
       [ 0.07284045,  0.32995867,  0.77647454, ..., -3.5575485 ,
        -2.82991263, -2.10169168],
       [ 0.07273468,  0.32464202,  0.74818458, ..., -2.89301525,
        -2.7038814 , -2.54436905],
       ...,
       [ 0.06353644,  0.22366663,  0.30310263, ...,  1.97122517,
         0.16034405, -1.98479915],
       [ 0.07250528,  0.26770499,  0.40934381, ...,  0.8185647 ,
         0.42084132, -0.41161594],
       [ 0.03869447,  0.10128017,  0.07158081, ...,  0.03039028,
        -1.93255217, -3.10721516]])

In [15]:
# Para guardar el filtrado frecuencial: 
import scipy.io as sio
import os
def create_mat_output_bandpass(output_task, output_data, file_name, folder, win_number = ""):
    mdic = {"task": output_task, "data":output_data}
    try:
        os.stat("./RegistrosFiltroPasoBanda/"+ folder)
    except:
        os.mkdir("./RegistrosFiltroPasoBanda/"+ folder)
    session = "./RegistrosFiltroPasoBanda/"+ folder + "/"+ file_name +"_passbandfilter_"+str(win_number)+".mat"
    sio.savemat(session, {'session':{'task_EEG':output_task, 'data_preprocessed_EEG':output_data, 'win_number': win_number}})

In [16]:
create_mat_output_bandpass(rsession.task, data_filtered, rsession.file_name, rsession.folder)
print("done user "+ str(rsession.user_id) )

done user W29


In [17]:
rsession.file_name

'W29_20210329_openloop_01'

## Se continúa con el Filtrado espacial 

Se comienza aplicando las Common Spatial Patterns. Para ello se va a estimar la transformada de las Common Spatial Patterns (CSP)

In [18]:
# Clase para crear objetos con los parámetros CSP (objetos processing)
class RProcessing:
    def __init__(self, fbands, channel_selection_names, channel_selection, m, W):
        self.fbands = fbands
        self.channel_selection_names = channel_selection_names
        self.channel_selection = channel_selection
        self.m = m
        self.W = W

# Clase para crear un objeto para añadir todos los archivos necesarios para calcular CSP.W
class RFiles:
    def __init__(self, filenames, pathname, n_files):
        self.filenames = filenames
        self.pathname = pathname
        self.n_files = n_files
        
# Clase para crear objetos de los datos ya filtrados con el bandpass filter        
class RSessionBandpass:
    def __init__(self,data_preprocessed_EEG,task_EEG, win_number):
        self.data_preprocessed_EEG = data_preprocessed_EEG
        self.task_EEG = task_EEG
        self.win_number = win_number

In [19]:
import scipy.io as sio
def read_record_bandpass(rec):
    ''' read_record('./RegistrosFiltroPasoBanda/W29_20210329_openloop_01_passbandfilter.mat')'''
    mat = sio.loadmat(rec)
    mdata = mat['session']
    val = mdata[0,0]
    rsessionBandpass = RSessionBandpass(val["data_preprocessed_EEG"], val["task_EEG"], val["win_number"])
    return rsessionBandpass

In [20]:
# Obtiene los canales a analizar
def match_electrodes(general_names, specific_names):
    electrodes_numbers = [0]*len(specific_names)
    for a in range(len(specific_names)):
        for b in range(len(general_names)):
            if specific_names[a]==general_names[b]:
                electrodes_numbers[a] = b
    return electrodes_numbers
    

In [21]:
import tkinter as tk
from tkinter import filedialog

def get_dummy_file_names():

    root = tk.Tk()
    root.withdraw()

    file_paths = filedialog.askopenfilenames(parent=root,title='Choose the EEG data files to create the model (cancel if it is the first trial)')
    file_paths_lists = list(file_paths)

    #file_path = "./"+file_paths_lists[0].split("/")[-2]
    file_path = file_paths_lists[0].split("/")[-2]
    
    file_names = []

    for i in file_paths_lists:
        file_names.append(i.split("/")[-1])
    
    n_files = len(file_names)
    return RFiles(file_names, str(file_path), n_files)

In [22]:
import scipy.sparse.linalg as sla
import scipy.linalg as sl
from numpy.linalg import matrix_rank, inv

def csp(X1, X2):
    """
    Function 'csp' trains a Common Spatial Pattern (CSP) filter bank.
    Input: 
        - X1 : Signal for the positive class. List with as many cells as N (no. trials) and each cell has dimensions [CxT]
        Where C is the no. channels and T the no. samples.
        - X2 : Signal for the negative class. List with as many cells as N (no. trials) and each cell has dimensions [CxT]
        Where C is the no. channels and T the no. samples.
        - Example: csp(data_previous_trials_fbands_class1[f][:],data_previous_trials_fbands_class2[f][:])
    Output: 
        - W : Filter matrix (mixing matrix, forward model). Note that the columns of W are the spatial filters.
    
    """
    # function CSP() from new_european
    # same formula as in the paper 'Optimal spatial filtering of single trialEEG during imagined hand movement' but for covariance matrix computation, +1e-20
    
    ####
    # Covariance matrix from each class
    ####
    
    N1 = len(X1) # Number of trials class 1
    C1a = [0]*N1
    for i in range(N1):
        C1a[i] = np.dot(X1[i],X1[i].transpose())/(np.dot(X1[i],X1[i].transpose())+1e-20).trace()
        #print(np.shape(C1a[i]))
        #print(C1a[i])
        
    N2 = len(X2) # Number of trials class 2
    C2a = [0]*N2
    for i in range(N1):
        C2a[i] = np.dot(X2[i],X2[i].transpose())/(np.dot(X2[i],X2[i].transpose())+1e-20).trace()
        #print(np.shape(C2a[i]))
        #print(C2a[i])
    
    #print(np.shape(C1a), np.shape(C2a))
    
    C1 = np.mean(C1a, axis=0) # mean of covariance matrixes for class 1, C1~[C x C]
    #print(np.shape(C1))
    #print(C1)
    C2 = np.mean(C2a, axis=0) # mean of covariance matrixes for class 2, C2~[C x C]
    #print(np.shape(C2))
    #print(C2)
    
    ####
    # Composite spatial covariance
    ####
    
    Cc = C1+C2
    #print(np.shape(Cc))
    #print(Cc)
    
    # A*v = lambda*v
    D, V = sla.eigs(Cc, matrix_rank(Cc)) #Subset of eigenvalues (v € R^{mxm}, D, diagonal matrix) and eigenvectors (lambda € R^{mx1}, V, colum vector)
    D = np.diag(D)
    #print(D)
    #print(V)
    P = np.dot(sl.sqrtm(inv(D)),V.transpose()) # Matrix square root.
    #print(np.shape(P))
    #print(P)
    S1 = np.dot(np.dot(P, C1),P.transpose())
    #print(np.shape(S1))
    #print(S1)
    S2 = np.dot(np.dot(P, C2),P.transpose())
    #print(np.shape(S2))
    #print(S2)
    
    D1, V1 = sla.eigs(S1, np.shape(S1)[1]) #Subset of eigenvalues (v € R^{mxm}, D, diagonal matrix) and eigenvectors (lambda € R^{mx1}, V, colum vector)
    D1 = np.diag(D1)
    D2, V2 = sla.eigs(S2, np.shape(S2)[1]) # it must be D1+D2=I and V1=V2
    D2 = np.diag(D2)
    I = D1+D2
    # print(I) 
    #print(np.shape(V1), np.shape(V2))
    #print(V1)
    #print(V2)
    # D1+D2=I and V1=V2 OKKKK
    
    W = np.dot(V2.transpose(),P)
    
    return W
    

In [23]:
def compute_FBCSP_transformation(channels, fbands, class1, class2, number_channels, folder =""):
    """
    Load together data from previous trials:
        Path /Sessions and /User name
        Select files that will be used to compute CSP transformation (W) at different frequency ranges
    Input: 
        - channels: channels selected to analyze.
        - fbands: number of frequency bands to analyze. Signal is separated in these frequency bands using band-pass filter in the
            preprocessing phase. NOTE: this number must meet the number of band-pass filters
        - class1 and class2: classes to separate using CSP (labels like 402,404).
    Output: 
        - W: CSP transformation. Cell structure, one cell element per frequency band
    """
    ####
    # 1º - Get the bandpass filter data (selected by the user):
    ####
    
    
    
    try:
        # If there are sessions of this subject
        files = get_dummy_file_names()
        #print(files.pathname, files.filenames, files.n_files )
        # Initialize 
        data_previous_trials_class1 = [0]*files.n_files
        data_previous_trials_class2 = [0]*files.n_files
        
        for i in range(files.n_files):
            
            # Read Record: 
            #record = str("./RegistrosFiltroPasoBanda/"+files.pathname+"/"+files.filenames[i])
            record = str("./RegistrosFiltroPasoBanda/"+folder+files.pathname+"/"+files.filenames[i])
            #print(record)
            rsessionBandpass = read_record_bandpass(record)
            #print(np.shape(rsessionBandpass.data_preprocessed_EEG))
            #print(rsessionBandpass.data_preprocessed_EEG)
            
            # Take data only with the desired classes
            
            i_class1 = [i[1] for i in np.argwhere(rsessionBandpass.task_EEG==class1)]
            #print(i_class1)
            i_class2 = [i[1] for i in np.argwhere(rsessionBandpass.task_EEG==class2)]
            #print(i_class2)
                
                # matrix with filtered data in different bands
            data_previous_trials_class1[i] = rsessionBandpass.data_preprocessed_EEG[:,i_class1]
            #print(np.shape(data_previous_trials_class1[i]))
            #print(data_previous_trials_class1[i])
            
            data_previous_trials_class2[i] = rsessionBandpass.data_preprocessed_EEG[:,i_class2]
            #print(np.shape(data_previous_trials_class2[i]))
            #print(data_previous_trials_class2[i])
            if i == 0: 
                n_rows_c = np.shape(data_previous_trials_class1[i])[0]
    except:
        print("Subject has not previous trials to create the CSP transformation")
    


    ####
    # 2º - Divide in frequency bands:
    ####

    # Initialize 
    data_previous_trials_fbands_class1 = []
    data_previous_trials_fbands_class2 = []
    for i in range(fbands):
        data_previous_trials_fbands_class1.append([0]*files.n_files)
        data_previous_trials_fbands_class2.append([0]*files.n_files)

    # Data preprocessed has the following size: [nºpreprocessing steps *nºchannels, nºsamples]
    n_rows = n_rows_c # number of rows of data_preprocessed, same for all trials and classes

    for f in range(fbands): 
        for j in range(files.n_files): 
            # Get the data from each bandpass filter:
            data_previous_trials_fbands_class1[f][j] = data_previous_trials_class1[j][n_rows-(fbands*number_channels)+((f+1-1)*number_channels):n_rows-(fbands*number_channels)+((f+1)*number_channels),:]
            data_previous_trials_fbands_class2[f][j] = data_previous_trials_class2[j][n_rows-(fbands*number_channels)+((f+1-1)*number_channels):n_rows-(fbands*number_channels)+((f+1)*number_channels),:]
            #print(data_previous_trials_fbands_class1[f][j])
            # Get the data from only channels selected
            data_previous_trials_fbands_class1[f][j] = data_previous_trials_fbands_class1[f][j][channels,:]
            data_previous_trials_fbands_class2[f][j] = data_previous_trials_fbands_class2[f][j][channels,:]
            #print(data_previous_trials_fbands_class1[f][j])
    #print(np.shape(data_previous_trials_fbands_class1))
    #print(data_previous_trials_fbands_class1)
    #print(np.shape(data_previous_trials_fbands_class2))
    #print(data_previous_trials_fbands_class2)

    ####
    # 3º - Compute CSP transform
    ####

    W_CSP = [0]*fbands
    for f in range(fbands):
        # Compute transformation matrix
        W_CSP[f] = csp(data_previous_trials_fbands_class1[f][:],data_previous_trials_fbands_class2[f][:])

    return W_CSP

In [24]:
# PARÁMETROS
fbands = 4 # Número de bandas de frecuencia a analizar 

channel_selection_names = ['F3', 'FZ', 'FC1','FCZ','C1','CZ','CP1','CPZ', 'FC5', 'FC3','C5','C3','CP5','CP3','P3','PZ','F4','FC2','FC4','FC6','C2','C4','CP2','CP4','C6','CP6','P4'] # Todos los electrodos seleccionados menos los de los ojos ['HR' ,'HL', 'VU', 'VD']

channel_selection = match_electrodes(electrodes_names_selected, channel_selection_names) # Posiciones de los electrodos seleccionados 

m = 4 # Características más importantes a analizar

# ¿¿¿¿¿¿ Esto es mejor hacerlo una vez para calcular W y luego utilizar la misma para todas las ventanas.
W = compute_FBCSP_transformation(channel_selection, fbands, 402 , 404, number_channels) #input: canales seleccionados para analizar, el nº de bandas de frecuencia seleccionadas, etiquetas de dos clases a distinguir, y el nº de canales


C:\Users\Sandrus\anaconda3\lib\site-packages\scipy\sparse\linalg\eigen\arpack\arpack.py:1267: RuntimeWarning: k >= N - 1 for N * N square matrix. Attempting to use scipy.linalg.eig instead.
  warnings.warn("k >= N - 1 for N * N square matrix. "


In [25]:
print(np.shape(W))
W[0]

(4, 27, 27)


array([[ 2.00528879e+00+0.j, -1.42485536e+00+0.j,  7.03298510e-01+0.j,
         1.05109493e+01+0.j, -3.41935976e+00+0.j, -6.29518832e+00+0.j,
         1.27201489e+00+0.j, -1.00020502e+01+0.j, -5.89690958e-01+0.j,
        -5.05696263e+00+0.j,  2.56074338e+00+0.j,  3.93874744e+00+0.j,
        -2.56371491e+00+0.j, -3.88644300e-01+0.j, -8.49738070e-01+0.j,
         5.74675034e+00+0.j, -4.32961390e+00+0.j, -4.87397375e+00+0.j,
         9.08963388e+00+0.j,  1.13666108e+00+0.j,  2.27284259e+00+0.j,
         1.68838739e+00+0.j,  5.00385728e+00+0.j, -3.40234051e+00+0.j,
        -4.56492928e+00+0.j,  2.16013593e+00+0.j, -1.37739599e+00+0.j],
       [ 1.90411877e+00+0.j, -2.67441730e-01+0.j, -2.27078612e+00+0.j,
        -9.26276225e+00+0.j,  2.19424060e+00+0.j, -3.00043700e+00+0.j,
        -3.46667552e-01+0.j, -3.02626183e+00+0.j,  1.64664017e+00+0.j,
         1.31933744e+00+0.j, -1.05404144e+01+0.j,  7.60146353e+00+0.j,
         2.57997833e+00+0.j, -3.82166026e+00+0.j,  3.82668269e+00+0.j,
     

In [26]:
processing = RProcessing(fbands, channel_selection_names, channel_selection, m, W)

In [27]:
processing.fbands

4

## Feature extraction

In [28]:
def features_FBCSP_MI(data_preprocessed, processing, number_channels):
    """
    Extracts the features of the channels included in input
    This function performs the following:
        - Extract CSP features at different frequency bands
        - output size is the number of frequency bands to analyze x 2 x m
    Input: 
        - data_EEG_preprocessed: matrix[channels, samples of a window] (or processing shift in the first epoch)
        - processing : CSP params 
        - number_channels
    Output:     
        - data_processed: matrix[channels, number of features per epoch]
    
    """
    ####
    # Output initialization: 
    ####
    output = np.zeros((processing.fbands*2*processing.m,1))
    ####
    # Apply
    ####
    n_rows = np.shape(data_preprocessed)[0]
    n_ch = number_channels
    
    for f in range(processing.fbands):
        # Apply transformation to each frequency band
        data = data_preprocessed[n_rows-(processing.fbands*number_channels)+((f+1-1)*number_channels):n_rows-(processing.fbands*number_channels)+((f+1)*number_channels),:]
        
        # Only to channels selected in conf.processing
        data_channels_selected = data[processing.channel_selection,:]
        
        # Z signal space filter => Z=W*E
        Z = np.dot(processing.W[f],data_channels_selected)
        
        # Only consider the variance of the m signals most suitable for discrimination (first m and last m rows of Z).
        variance_Zp = np.zeros((1,2*processing.m))
        Zp = np.concatenate((Z[0:processing.m,:], Z[-(processing.m+1):-1,:]), axis=0)
        
        for p in range(2*processing.m):
            variance_Zp[0,p] = np.var(Zp[p,:])
        
        # Normalize variance of each signal
        output[(f)*2*processing.m:(f+1)*2*processing.m,:] = np.log10(variance_Zp.transpose()/variance_Zp.sum(axis=1))
    #print(np.shape(output))
    #print(output)
    
    return output

In [29]:
output_features_FBCSP_MI = features_FBCSP_MI(data_filtered, processing, number_channels)
print(np.shape(output_features_FBCSP_MI))
output_features_FBCSP_MI

(32, 1)


array([[-0.88368556],
       [-0.92875372],
       [-0.8995839 ],
       [-0.90434164],
       [-0.88716497],
       [-0.92482833],
       [-0.91566957],
       [-0.88335802],
       [-0.8827804 ],
       [-0.9175893 ],
       [-0.87992665],
       [-0.88775991],
       [-0.92202661],
       [-0.8769376 ],
       [-0.9524826 ],
       [-0.91082525],
       [-0.9558535 ],
       [-0.93620898],
       [-0.86621201],
       [-0.8808721 ],
       [-0.90926948],
       [-0.91530328],
       [-0.91093421],
       [-0.85915006],
       [-0.88702471],
       [-0.88952345],
       [-0.92659199],
       [-0.91631476],
       [-0.93583321],
       [-0.87555525],
       [-0.89536829],
       [-0.90201299]])

# 2- Preprocesado, procesado y extracción de características de todos los datasets con  WINDOWING:

Vamos a intentar sacar la W, una por cada .mat 

In [30]:
def preprocessing(data, fm, number_channels):
    
    filt_order = 2 # Todos los filtros tienen el mismo orden. Serán filtros de 2º orden Butterworth
    # Primer filtro: 
    low_f = 5
    high_f = 10
    data_filter1 = butter_bandpass_filter(data, fm, number_channels, filt_order, low_f, high_f)
    data_filter1


    # Segundo filtro: 
    low_f = 10
    high_f = 15
    data_filter2 = butter_bandpass_filter(data, fm, number_channels, filt_order, low_f, high_f)
    data_filter2

    # Tercer filtro: 
    low_f = 15
    high_f = 20
    data_filter3 = butter_bandpass_filter(data, fm, number_channels, filt_order, low_f, high_f)
    data_filter3

    # Cuarto filtro: 
    low_f = 20
    high_f = 25
    data_filter4 = butter_bandpass_filter(data, fm, number_channels, filt_order, low_f, high_f)
    data_filter4

    data_filtered = np.concatenate((np.array(data_filter1), np.array(data_filter2)), axis = 0)
    data_filtered = np.concatenate((np.array(data_filtered), np.array(data_filter3)), axis = 0)
    data_filtered = np.concatenate((np.array(data_filtered), np.array(data_filter4)), axis = 0)
    
    return data_filtered

In [31]:
def SACAR_W(session, number_channels, fm, channel_selection, fbands, task1 , task2):
    print("PREPROCESSING")
    data_filtered = preprocessing(session.data, fm, number_channels)
    create_mat_output_bandpass(session.task, data_filtered, session.file_name, session.folder + "/"+session.file_name, -1)    # To use to compute W        
    print("Done")
    print("PROCESSING")
    W = compute_FBCSP_transformation(channel_selection, fbands, task1 , task2, number_channels, session.folder+"/" ) #input: canales seleccionados para analizar, el nº de bandas de frecuencia seleccionadas, etiquetas de dos clases a distinguir, y el nº de canales
    processing = RProcessing(fbands, channel_selection_names, channel_selection, m, W)
    return processing

In [32]:
# Para guardar las características: 
import scipy.io as sio
def create_mat_processing_W(processing, file_name, folder):
    session = "./W_procesado/"+ folder + "/"+ file_name +"_processingW.mat"
    sio.savemat(session, {'processing':{'fbands':processing.fbands, 'channel_selection_names':processing.channel_selection_names, 
                                        'channel_selection':processing.channel_selection, 'm':processing.m, 'W':processing.W}})
    

In [33]:
# Parámetros globales
fm = 200 # Frecuencia de Muestreo - Hz
electrodes_names_selected = ['F3', 'FZ', 'FC1','FCZ','C1','CZ','CP1','CPZ', 'FC5', 'FC3','C5','C3','CP5','CP3','P3',
                             'PZ','F4','FC2','FC4','FC6','C2','C4','CP2','CP4','C6','CP6','P4','HR' ,'HL', 'VU', 'VD']
electrodeEyes_names_selected = ['HR', 'HL', 'VU', 'VD'] # Electrodos que se colocarón en los ojos - No los utilizaremos 
number_channels = len(electrodes_names_selected)
#print(number_channels)

# PARÁMETROS INDIVIDUALES
fbands = 4 # Número de bandas de frecuencia a analizar 

channel_selection_names = ['F3', 'FZ', 'FC1','FCZ','C1','CZ','CP1','CPZ', 'FC5', 'FC3','C5','C3','CP5','CP3','P3','PZ','F4','FC2','FC4','FC6','C2','C4','CP2','CP4','C6','CP6','P4'] # Todos los electrodos seleccionados menos los de los ojos ['HR' ,'HL', 'VU', 'VD']

channel_selection = match_electrodes(electrodes_names_selected, channel_selection_names) # Posiciones de los electrodos seleccionados 

m = 4 # Características más importantes a analizar

records = ["W29-29_03_2021", "W29-31_03_2021", "W29-01_04_2021"]
#records = ["W29-29_03_2021"]
number_records_mat = [22, 26, 22]
record_contain_name = ["20210329", "20210331", "20210401"]
for i in range(len(records)):  
    print(records[i])
    for j in range(1,number_records_mat[i]+1):
        k = j 
        if k<10: 
            k = "0"+str(k) 
        rsession = read_record('./RegistrosSinProcesar/'+records[i]+'/W29_'+record_contain_name[i]+'_openloop_'+str(k)+'.mat')
        processing  = SACAR_W(rsession, number_channels, fm, channel_selection, fbands, 402 , 404)
        create_mat_processing_W(processing, rsession.file_name , rsession.folder)
        print("Done ",rsession.file_name)
        print()
        
        # SELECCIONAR LOS QUE SON -1 POR EJEMPLO W29_20210329_openloop_01_passbandfilter_-1

W29-29_03_2021
PREPROCESSING
Done
PROCESSING
Subject has not previous trials to create the CSP transformation


UnboundLocalError: local variable 'files' referenced before assignment

### Ahora ya con la W calculada vamos a preprocesar, procesar y extraer las características

In [39]:
import scipy.io as sio
def read_processing_W(rec):
    ''' read_record('./RegistrosFiltroPasoBanda/W29_20210329_openloop_01_passbandfilter.mat')'''
    mat = sio.loadmat(rec)
    mdata = mat['processing']
    val = mdata[0,0]
    rprocessing = RProcessing(int(val["fbands"]), val["channel_selection_names"], val["channel_selection"], int(val["m"]), val["W"])
    return rprocessing


In [40]:
def FBCSP_WCALCULADA(session, number_channels, fm, channel_selection, fbands, task1 , task2, pathW):
    
    window = 300
    samples_advance = 100 
    
    output_task = []
    output_data = []
    
    data_filtered_l = []
    
    win_init = int(0)
    win_number = 1
    
    print("PREPROCESSING")
    for i in range(np.shape(session.data)[1]): # For each signal registered
        win_end = int(win_init + window)
        if win_end >= np.shape(session.data)[1]:
            break
        
        task = np.unique(session.task[0,win_init:win_end])
        #task = session.task[0,win_init:win_end]
        
        if len(task)==1:
        #if task1 in task or task2 in task:
            signal_window = session.data[:,win_init:win_end]
            data_filtered = preprocessing(signal_window, fm, number_channels)
            data_filtered_l.append(data_filtered)
            create_mat_output_bandpass(task, data_filtered, session.file_name, session.folder + "/"+session.file_name, win_number)    # To use to compute W        
            win_number += 1
            
        win_init += int(samples_advance)
    print("Done")
    print("PROCESSING")
    processing = read_processing_W(pathW)
    W = processing.W
    
    print("Done")
    print("FEATURE EXTRACTION")
    win_init = int(0)
    window_position = 0
    for i in range(np.shape(session.data)[1]): # For each signal registered
        win_end = int(win_init + window)
        if win_end >= np.shape(session.data)[1]:
            break
        
        task = np.unique(session.task[0,win_init:win_end])
        #task = session.task[0,win_init:win_end]
        
        if len(task)==1:
        #if task1 in task or task2 in task:
            data_filtered = data_filtered_l[window_position]
            output_features_FBCSP_MI = features_FBCSP_MI(data_filtered, processing, number_channels)

            output_task.append(task)
            output_data.append(output_features_FBCSP_MI)

            window_position+=1
            
        win_init += int(samples_advance)
    print("Done")
    return output_task, output_data, processing  

In [41]:
# Para guardar las características: 
import scipy.io as sio
def create_mat_output_features_2(output_task, output_data, file_name, folder, W):
    mdic = {"task": output_task, "data":output_data}
    session = "./RegistrosProcesados2/"+ folder + "/"+ file_name +"_processed.mat"
    sio.savemat(session, {'session':{'task_EEG_p':output_task, 'data_processed_EEG':output_data, 'W':W}})
    

In [42]:
# Parámetros globales
fm = 200 # Frecuencia de Muestreo - Hz
electrodes_names_selected = ['F3', 'FZ', 'FC1','FCZ','C1','CZ','CP1','CPZ', 'FC5', 'FC3','C5','C3','CP5','CP3','P3',
                             'PZ','F4','FC2','FC4','FC6','C2','C4','CP2','CP4','C6','CP6','P4','HR' ,'HL', 'VU', 'VD']
electrodeEyes_names_selected = ['HR', 'HL', 'VU', 'VD'] # Electrodos que se colocarón en los ojos - No los utilizaremos 
number_channels = len(electrodes_names_selected)
#print(number_channels)

# PARÁMETROS INDIVIDUALES
fbands = 4 # Número de bandas de frecuencia a analizar 

channel_selection_names = ['F3', 'FZ', 'FC1','FCZ','C1','CZ','CP1','CPZ', 'FC5', 'FC3','C5','C3','CP5','CP3','P3','PZ','F4','FC2','FC4','FC6','C2','C4','CP2','CP4','C6','CP6','P4'] # Todos los electrodos seleccionados menos los de los ojos ['HR' ,'HL', 'VU', 'VD']

channel_selection = match_electrodes(electrodes_names_selected, channel_selection_names) # Posiciones de los electrodos seleccionados 

m = 4 # Características más importantes a analizar

records = ["W29-29_03_2021", "W29-31_03_2021", "W29-01_04_2021"]
#records = ["W29-29_03_2021"]
number_records_mat = [22, 26, 22]
record_contain_name = ["20210329", "20210331", "20210401"]
for i in range(len(records)):  
    print(records[i])
    for j in range(1,number_records_mat[i]+1):
        k = j 
        if k<10: 
            k = "0"+str(k) 
        rsession = read_record('./RegistrosSinProcesar/'+records[i]+'/W29_'+record_contain_name[i]+'_openloop_'+str(k)+'.mat')
        
        pathW = "./W_procesado/"+records[i]+'/W29_'+record_contain_name[i]+"_openloop_"+str(k)+'_processingW.mat'
    
        output_task, output_data, processing  = FBCSP_WCALCULADA(rsession, number_channels, fm, channel_selection, fbands, 402 , 404, pathW)
        create_mat_output_features_2(output_task, output_data, rsession.file_name , rsession.folder, processing.W)
        print("Done ",rsession.file_name)
        print()

W29-29_03_2021
PREPROCESSING
Done
PROCESSING
Done
FEATURE EXTRACTION
Done
Done  W29_20210329_openloop_01

PREPROCESSING
Done
PROCESSING
Done
FEATURE EXTRACTION
Done
Done  W29_20210329_openloop_02

PREPROCESSING
Done
PROCESSING
Done
FEATURE EXTRACTION
Done
Done  W29_20210329_openloop_03

PREPROCESSING
Done
PROCESSING
Done
FEATURE EXTRACTION
Done
Done  W29_20210329_openloop_04

PREPROCESSING
Done
PROCESSING
Done
FEATURE EXTRACTION
Done
Done  W29_20210329_openloop_05

PREPROCESSING
Done
PROCESSING
Done
FEATURE EXTRACTION
Done
Done  W29_20210329_openloop_06

PREPROCESSING
Done
PROCESSING
Done
FEATURE EXTRACTION
Done
Done  W29_20210329_openloop_07

PREPROCESSING
Done
PROCESSING
Done
FEATURE EXTRACTION
Done
Done  W29_20210329_openloop_08

PREPROCESSING
Done
PROCESSING
Done
FEATURE EXTRACTION
Done
Done  W29_20210329_openloop_09

PREPROCESSING
Done
PROCESSING
Done
FEATURE EXTRACTION
Done
Done  W29_20210329_openloop_10

PREPROCESSING
Done
PROCESSING
Done
FEATURE EXTRACTION
Done
Done  W29_202103